# Streaming

***1. “DealExtreme” es una empresa que vende una enorme cantidad de productos importados de bajo costo
de China. Se analiza un stream en donde se recibe el ID de cada producto vendido. Cada vez que se venden 100 unidades de un determinado producto, el departamento de marketing publica un pequeño aviso sobre el mismo. Esto permite aumentar aún más las ventas de los productos populares.***

***A la gente de marketing se le ocurrió también que sería una buena idea publicitar los productos que no han llegado nunca a las 100 ventas. Para ello, crearon una campaña en la cual crean un aviso especial para los productos vendidos los días viernes que nunca tuvieron publicado un aviso.***

***Diseñar una solución que, a través del procesamiento del stream, permita a la gente de marketing
determinar si debe publicar un aviso o no en base al ID del producto.***

***Observación: notar que no interesa la cantidad total de unidades vendidas para cada producto, sino
simplemente la detección de cuándo la cantidad de ventas llega a 100.***

A medida que van llegando los ids de los productos vendidos, usamos un `bloom filter` para guardarse en un vector binario la información de si un producto fue efectivamente publicitados o no. La probabilidad de tener un falso positivo y negativo se calcularán en función de la cantidad de hashes a usar, etc.

Luego para verificar la cantidad de ventas usamos un Count-Min, donde nos guardamos en muchos vectores los contadores en los índices de los hashes del elemento. 

***2. Dado un stream compuesto por los siguientes números: 3, 1, 4, 1, 5, 9, 2, 6, 5; se quiere aplicar el algoritmo Flajolet-Martin para calcular el momento de orden 0 del stream usando una función de hashing de la familia: h(x) = ax + b mod 32. El resultado debe tomarse como un número de 5 bits contando la cantidad de ceros a derecha del mismo. No todos los valores de a y b son adecuados, por lo que se debe explicar qué valores son los más adecuados y por qué. A modo de ejemplo, analizar las funciones resultantes de usar: a = 2, b = 1; a = 3, b = 7; y a= 4, b = 0.***

Tenemos la siguiente data:

In [5]:
stream = [3, 1, 4, 1, 5, 9, 2, 6, 5]
h = lambda a,b,x: (a*x + b) % 32

El momento de orden cero del stream es efectivamente 7. Esto se puede aproximar con Flajolet-Martin.
Probemos las diferentes combinaciones:

In [13]:
a_usar = ((2,1), (3,7), (4,0))
for tupla in a_usar:
    a , b = tupla
    print(f"\na = {a}, b = {b}")
    for s in stream:
        nro = "{:05b}".format(h(a,b,s))
        print(f"h({s}) = {nro}")


a = 2, b = 1
h(3) = 00111
h(1) = 00011
h(4) = 01001
h(1) = 00011
h(5) = 01011
h(9) = 10011
h(2) = 00101
h(6) = 01101
h(5) = 01011

a = 3, b = 7
h(3) = 10000
h(1) = 01010
h(4) = 10011
h(1) = 01010
h(5) = 10110
h(9) = 00010
h(2) = 01101
h(6) = 11001
h(5) = 10110

a = 4, b = 0
h(3) = 01100
h(1) = 00100
h(4) = 10000
h(1) = 00100
h(5) = 10100
h(9) = 00100
h(2) = 01000
h(6) = 11000
h(5) = 10100


Entonces, el algoritmo dice que la máxima cantidad de ceros a la derecha que podamos encontrar, representa la variable $r$, y según el algorítmo $2^r = M^0(S)$.

En el caso de $a = 2, b = 1$ tenemos que $r = 0$ ya que la máxima cantidad de ceros que podemos encontrar a la derecha de cualquier hash, siempre va a ser nula. Por lo tanto estos coeficientes aproximan a $M^0(S) = 2^0 = 1$.

En los otros dos casos $r = 4 \implies M^0(S) = 2^4 = 16$ lo cual sigue siendo incorrecto. 

Para encontrar el resultado correspondiente debemos pedir que la diferencia entre nuestra aproximación y el momento real sea mínima, y la potencia de 2 más cercana a 7 es 8. Por lo tanto queremos que $r = 3$ y eso se logra solamente si nuestra máxima cantidad de ceros a la derecha es 3.

CASUALMENTE existe una solución con $(a,b) = (2,2)$ que asegura que $r = 3$

***3. Se desea utilizar Flajolet-Martin para estimar la cantidad de elementos distintos en un stream. Suponer que de los 10 elementos posibles (números naturales del 1 al 10), solo 4 de ellos aparecen efectivamente en el stream. Para realizar la estimación, se puede hashear cada elemento en binarios de 4 bits con h(x) = 3x + 7 mod 11. Por ejemplo, para x = :, h(x) = 31 mod 11 = 9, por lo que en binario: h(x) = 1001. A los efectos de la implementación, contar los ceros del final del código (por ejemplo, para “0100”, son 2 ceros). Encontrar todos los subconjuntos de 4 elementos que hacen que la estimación sea exacta.***

Sabemos que $M^0(S) = 4 = 2^r$ por lo que debemos tener un hash que genere dos ceros al final.

In [39]:
h2 = lambda x: (3*x + 7) % 11

In [43]:
for i in range(1,11):
    valor = "{:05b}".format(h2(i))
    print(f"i={i} -> h(i)={valor}")

i=1 -> h(i)=01010
i=2 -> h(i)=00010
i=3 -> h(i)=00101
i=4 -> h(i)=01000
i=5 -> h(i)=00000
i=6 -> h(i)=00011
i=7 -> h(i)=00110
i=8 -> h(i)=01001
i=9 -> h(i)=00001
i=10 -> h(i)=00100


Sabemos que el 10 tiene que estar pues cumple que $r = 2$ y que el $4,5$ no pueden estar pues hacen que crezca $r$. Por lo tanto nuestro conjunto está compuesto por cualquier set de 3 números que no tengan el 4,5 y unión con el 10.

***4. Aplicando Reservoir Sampling con k = 4, se observa el siguiente stream: 5, 6, 7, 8. A continuación, se observa el elemento 3. Justificar cuál es la probabilidad de que 6 continúe en la muestra de Reservoir Sampling.***

La probabilidad de que el 6 siga disponible es equivalente a pedir que un número aleatorio entre 0,1 (con distribución uniforme) sea menor a $\frac{k}{n} \implies p = \frac{4}{5}$. 

***5. a. Construir un filtro de bloom de 16 bits (m = 16) que contenga los caracteres “C” y “D”, para el universo de 5 (n = 5) caracteres: “A”, “B”, “C”, “D” y “E” considerando las siguientes funciones de hashing “h1” y “h2”. Tener en cuenta que la probabilidad de falsos positivos para la construcción es de 0,2.***

|    | A  | B | C | D | E |
|----|----|---|---|---|---|
| h1 | 13 | 8 | 4 | 4 | 3 |
| h2 | 15 | 5 | 9 | 15| 7 |

***b. Sabiendo que el valor del “m” que se utilizó es óptimo, indicar si la cantidad de funciones de hashing utilizadas es óptima (justificar).***


La probabilidad de falsos positivos es $0.2$. Prendo los bits correspondientes a los índices que dan los hashes:

```
filtro = [0,0,0,1,0,0,0,0,1,0,0,0,0,0,1]
```

La cantidad de funciones de hashing se calcula con: $\lfloor \frac{16}{5} \cdot ln(2) \rfloor = 2$ por lo tanto estamos usando la cantidad correcta de hashes.

***6. Dados los siguientes streams:***

- E A B D C A E C B D

- E A E B C E E D B E

***estimar el número sorpresa utilizando AMS con 5 estimadores, y explicar qué conclusiones se pueden
obtener de la comparación de los resultados para ambos streams.***

Tenemos que estimar el número sorpresa con AMS. Este dice que $M^2(S) = (n(2C_i - 1))$ pero como la cantidad de estimadores es igual a la cantidad de elementos únicos que tenemos, por lo tanto la aproximación siempre será igual al momento de orden dos real.

***7. Se usa el algoritmo Count-Min con 3 filtros de 8 posiciones cada uno. El estado de los filtros está dado por:***

```
[0 2 4 2 0 0 3 5]
[1 0 2 6 1 2 0 4]
[0 0 3 3 3 2 4 1]
```

***Indicar cuáles de las siguientes afirmaciones son verdaderas (justificando adecuadamente).***

***a. La cantidad total de elementos del stream es 16:*** VERDADERO porque en los 3 vectores se incrementan 16 veces los valores de los mismos, para cada filtro.

***b. No puede tratarse de un stream con 16 elementos diferentes:*** FALSA pues los índices que nos dan los hashes pueden ser el producto de colisiones.

***c. Puede existir un elemento con frecuencia 5:*** FALSA pues ningun elemento del tercer vector es mayor o igual a 5, por lo que ningun elemento puede haber incrementado 5 o más veces alguna posición.

***8. ¿Cuál es la cantidad máxima que puede estimar el algoritmo Count-Min dados los siguientes filtros? ¿Qué resultados debería dar la función de hashing para lograr dicha estimación?***

```
[2 6 0 0 0 3 0 1]
[0 2 1 1 2 2 4 3]
[3 2 0 0 2 4 3 2]
```

Supongo que se refiere a 12 pues es la suma de numeros de los vectores xd

***9. Se usan 3 funciones de hashing 0..4 para el algoritmo Count-Min. Para las siguientes palabras, se da el resultado de las 3 funciones de hashing:***

***“casa” -> [2 1 2] “canasta” -> [0 3 3] “kilo” -> [4 1 0] “alfa” -> [3 3 0]***

***si los filtros son los siguientes:***
```
f1 = [0 2 3 1 3]
f2 = [1 2 2 4 0]
f3 = [3 0 1 1 1]
```
***¿Cuál de las cuatro palabras es estimada como la más frecuente?***

Los mínimos son:

```
min{f1[h(casa)],    f2[h(casa)],    f3[h(casa)]}    = 1
min{f1[h(canasta)], f2[h(canasta)], f3[h(canasta)]} = 0
min{f1[h(kilo)],    f2[h(kilo)],    f3[h(kilo)]}    = 1
min{f1[h(alfa)],    f2[h(alfa)],    f3[h(alfa)]}    = 1
```

Es un empate entre casa, kilo y alfa.

***10. Dado el siguiente stream: 1, 3, 2, 3, 3, 5, 2, 1; indicar una función de hashing de la forma h(x) = ax + b mod 32 tal que el algoritmo de Flajolet-Martin se aproxime de la mejor forma posible al cálculo del momento de orden 0 del stream. Usar 5 bits y tomar los bits a derecha para el algoritmo.***

Para mejor aproximar el calculo de orden 0 necesito que $r = 2$. Por ejemplo con $(a,b) = (1,2)$:

In [51]:
h10 = lambda x: (1*x + 2) % 32
stream10 = set([1, 3, 2, 3, 3, 5, 2, 1])
for i in stream10:
    nro = "{:05b}".format(h10(i))
    print(f"h({i}) = {nro}")

h(1) = 00011
h(2) = 00100
h(3) = 00101
h(5) = 00111


***11. Dado el siguiente stream: 1, 3, 4, 3, 4, 2, 2, 4, 2, 2; calcular el momento de orden 2 del stream y luego realizar una estimación usando AMS a partir de los valores del stream 2, 2, 4, 2, 2. Usar el promedio entre estimadores para la estimación final.***

El momento real del stream es:

In [52]:
sum([i**2 for i in [1, 3, 4, 3, 4, 2, 2, 4, 2, 2]])

83